<a href="https://colab.research.google.com/github/AnudeepReddy-Katta/SentimentAnalysis/blob/main/notebooks/Data_Preparation_and_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 1.0MB 8.7MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=df82490b553d0473b45971e2a141edf6b83dfb1e87be55dfb48e3389d153c956
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [ ]:
import pandas as pd
datasetSentences = pd.read_csv(("/content/drive/MyDrive/NLP/datasetSentences.txt"), sep="\t", skiprows=[0], names=['sentence_index','sentence'])
sentiment_labels = pd.read_csv(("/content/drive/MyDrive/NLP/sentiment_labels.txt"), sep="|", skiprows=[0], names=['phrase_id','sentiment_values'])
dictionary = pd.read_csv(("/content/drive/MyDrive/NLP/dictionary.txt"), sep="|", names=['phrase','phrase_id'])

In [ ]:
def pre_process_sentences(string):
  string=string.replace('-LRB-','(')
  string=string.replace('-RRB-',')')
  string=string.replace('Â', '')
  string=string.replace('Ã©', 'e')
  string=string.replace('Ã¨', 'e')
  string=string.replace('Ã¯', 'i')
  string=string.replace('Ã³', 'o')
  string=string.replace('Ã´', 'o')
  string=string.replace('Ã¶', 'o')
  string=string.replace('Ã±', 'n')
  string=string.replace('Ã¡', 'a')
  string=string.replace('Ã¢', 'a')
  string=string.replace('Ã£', 'a')
  string=string.replace('\xc3\x83\xc2\xa0', 'a')
  string=string.replace('Ã¼', 'u')
  string=string.replace('Ã»', 'u')
  string=string.replace('Ã§', 'c')
  string=string.replace('Ã¦', 'ae')
  string=string.replace('Ã­', 'i')
  string=string.replace('\xa0', ' ')
  string=string.replace('\xc2', '')
  return string

In [ ]:
datasetSentences['sentence'] = datasetSentences['sentence'].apply(pre_process_sentences)

In [ ]:
def pre_process_phrases(string):
    string=string.replace('é','e')
    string=string.replace('è','e')
    string=string.replace('ï','i')
    string=string.replace('í','i')
    string=string.replace('ó','o')
    string=string.replace('ô','o')
    string=string.replace('ö','o')
    string=string.replace('á','a')
    string=string.replace('â','a')
    string=string.replace('ã','a')
    string=string.replace('à','a')
    string=string.replace('ü','u')
    string=string.replace('û','u')
    string=string.replace('ñ','n')
    string=string.replace('ç','c')
    string=string.replace('æ','ae')
    string=string.replace('\xa0', ' ')
    string=string.replace('\xc2', '')    
    return string 

In [ ]:
dictionary['phrase'] = dictionary['phrase'].apply(pre_process_phrases)

In [ ]:
df1 = pd.merge(sentiment_labels,dictionary,on='phrase_id')
df1.drop(columns=['phrase_id'],inplace = True)
df1.head()

,sentiment_values,phrase
0,0.50000,!
1,0.50000,'
2,0.44444,' (
3,0.50000,' ( the cockettes
4,0.42708,' ( the cockettes )


In [ ]:
df2 = pd.merge(df1,datasetSentences,left_on='phrase',right_on='sentence')
df2.drop(columns = ['phrase','sentence_index'],inplace = True)
df2.head()

,sentiment_values,sentence
0,0.013889,... a bland murder-on-campus yawner .
1,0.263890,... a hollow joke told by a cinematic gymnast ...
2,0.472220,... the picture 's cleverness is ironically mu...
3,0.875000,classic cinema served up with heart and humor
4,0.319440,"entertaining enough , but nothing new"


In [ ]:
df2.loc[(df2['sentiment_values']>=0) & (df2['sentiment_values']<=0.2),'label']=0
df2.loc[(df2['sentiment_values']>0.2) & (df2['sentiment_values']<=0.4),'label']=1
df2.loc[(df2['sentiment_values']>0.4) & (df2['sentiment_values']<=0.6),'label']=2
df2.loc[(df2['sentiment_values']>0.6) & (df2['sentiment_values']<=0.8),'label']=3
df2.loc[(df2['sentiment_values']>0.8) & (df2['sentiment_values']<=1),'label']=4
df2.drop(columns = ['sentiment_values'],inplace = True)
df2.rename(columns={'label':'sentiment_values'},inplace = True)
df2['sentiment_values'] = df2['sentiment_values'].astype('int')

In [ ]:
df2['sentiment_values'].value_counts()

1    3140
3    3111
2    2242
4    1851
0    1510
Name: sentiment_values, dtype: int64

In [ ]:
len(df2)

11854

In [ ]:
def random_deletion(sentence, p=0.5): 
    words = sentence.split()
    if len(words) == 1: # return if single word
        return words[0]
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)][0]
    else:
        return ' '.join(remaining)

In [ ]:
def random_swap(sentence, n=5): 
    sentence = sentence.split()
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join(sentence)

In [ ]:
import random
import googletrans
from  googletrans import Translator

def back_translate(sentence):
  sentence = sentence.split()
  translator = Translator()
  available_langs = list(googletrans.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 

  translations = translator.translate(sentence, dest=trans_lang) 
  t_text = [t.text for t in translations]

  translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
  en_text = [t.text for t in translations_en_random]
  return ' '.join(en_text)

In [ ]:
from tqdm import tqdm
import numpy as np
from random import shuffle
def augment_text(df,samples,label,samples_back_translate=0.1,
                 samples_random_swap=.45,samples_random_deletion=.45):
    new_text=[]
    
    ##selecting a class samples
    df_n=df[df.sentiment_values==label].reset_index(drop=True)

    ## data augmentation loops
    for i in tqdm(np.random.randint(0,len(df_n),int(samples_back_translate*samples))):
        
            text = df_n.iloc[i]['sentence']
            augmented_text = back_translate(text)
            new_text.append(augmented_text)
    
    for i in tqdm(np.random.randint(0,len(df_n),int(samples_random_swap*samples))):
        
            text = df_n.iloc[i]['sentence']
            augmented_text = random_swap(text)
            new_text.append(augmented_text)

    for i in tqdm(np.random.randint(0,len(df_n),int(samples_random_deletion*samples))):
        
            text = df_n.iloc[i]['sentence']
            augmented_text = random_deletion(text)
            new_text.append(augmented_text)
    
    ## dataframe
    new=pd.DataFrame({'sentence':new_text,'sentiment_values':label})
    return new

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df2, test_size=0.2, random_state=42, stratify=df2['sentiment_values']) 

In [ ]:
df_0 = augment_text(train,1300,0)

100%|██████████| 585/585 [00:00<00:00, 6545.80it/s]


In [ ]:
df_0.to_csv('/content/drive/MyDrive/NLP/df_0.csv')

In [ ]:
df_2 = augment_text(train,700,2)

100%|██████████| 315/315 [00:00<00:00, 6409.39it/s]


In [ ]:
df_2.to_csv('/content/drive/MyDrive/NLP/df_2.csv')

In [ ]:
df_4 = augment_text(train,1000,4)

100%|██████████| 450/450 [00:00<00:00, 6106.63it/s]


In [ ]:
df_4.to_csv('/content/drive/MyDrive/NLP/df_4.csv')

In [ ]:
train.to_csv('/content/drive/MyDrive/NLP/train.csv')

In [ ]:
test.to_csv('/content/drive/MyDrive/NLP/test.csv')

In [ ]:
train = train.reset_index()
train.drop(columns='index',inplace=True)
train = train.append(df_0).reset_index(drop=True)
train = train.append(df_2).reset_index(drop=True)
train = train.append(df_4).reset_index(drop=True)

In [ ]:
train.to_csv('/content/drive/MyDrive/NLP/train_aug.csv')

In [ ]:
test =test.reset_index(drop=True)

In [ ]:
test.to_csv('/content/drive/MyDrive/NLP/test.csv')